<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_AviationQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import tensorboard

In [ ]:
# Install Pytorch & other libraries
#!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet


#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


In [ ]:
# versions: 0.0.1, 0.0.2, 0.0.3, 0.1.0, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.5.0, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.7.8, 0.7.9, 0.7.10, 0.7.11, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.8.6, 0.9.2, 0.9.3, 0.9.4)
#ERROR: No matching distribution found for trl==0.0.99
! pip install trl==0.8.6 -q

In [3]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [4]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [51]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

In [6]:
# set device
device = 'cuda'

In [7]:
torch.__version__

'2.3.0+cu121'

In [ ]:
#!apt-get update && apt-get install -y cuda-11-0

In [8]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Fri Jun  7 13:46:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   38C    P8       

In [9]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

{'prompt': '<prompt text>', 'completion': '<ideal generated text>'}

In [7]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"



https://huggingface.co/datasets/sakharamg/AviationQA

In [ ]:
from datasets import load_dataset
print("Preprocessing dataset AviationQA")
dataset = load_dataset("sakharamg/AviationQA")

dataset = dataset.map(create_prompt_formats)

# save datasets to disk
dataset["train"].to_json("train_dataset_AviationQA.json", orient="records")
dataset["validation"].to_json("validation_dataset_AviationQA.json", orient="records")
dataset["test"].to_json("test_dataset_AviationQA.json", orient="records")

In [9]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'Question', 'Answer'],
        num_rows: 1057986
    })
    validation: Dataset({
        features: ['id', 'Question', 'Answer'],
        num_rows: 11888
    })
    test: Dataset({
        features: ['id', 'Question', 'Answer'],
        num_rows: 10807
    })
})


In [10]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/train_dataset_AviationQA.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
print(dataset)

Dataset({
    features: ['id', 'Question', 'Answer'],
    num_rows: 1057986
})


In [12]:
dataset[2]

{'id': 'a529971724',
 'Question': 'Who determines the probable cause(s) of this accident no. ERA22LA104?',
 'Answer': 'The National Transportation Safety Board'}

In [13]:
nrec=10000
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['Question'][0:nrec]
dataset_final_Answer=dataset['Answer'][0:nrec]

In [14]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [40]:
datasetF

,id,Question,Answer,text
0,a9567233027,Where was the Destination of the accident no. ...,"Los Angeles, CA",Bob is a Traveller
1,a4671579659,What is the name of the Engine Manufacturer of...,Pratt & Whitney,Bob is a Traveller
2,a529971724,Who determines the probable cause(s) of this a...,The National Transportation Safety Board,Bob is a Traveller
3,a1197873694,What are the Probable findings of the accident...,Environmental issues Terrain induced turbulenc...,Bob is a Traveller
4,a7803769704,"In the accident with no. DCA16CA009, is the ai...",None,Bob is a Traveller
...,...,...,...,...
9995,a4688878518,Was there any Operating Certificate(s) held of...,None,Bob is a Traveller
9996,a8331736975,What was the VFR Approach/Landing of the accid...,Go around,Bob is a Traveller
9997,a4342209241,Was there fire on aircraft of the accident no....,None,Bob is a Traveller
9998,a2435551586,Who determines the probable cause(s) of this a...,The National Transportation Safety Board,Bob is a Traveller


No charts were generated by quickchart


In [16]:
datasetF['Question'][99]

'What was the Temperature/Dew Point of the accident no. CEN16LA033?'

In [19]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
)

In [20]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            52Gi       2.1Gi       5.7Gi       6.0Mi        45Gi        50Gi
Swap:             0B          0B          0B


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1" #01 march 2024, 08 MARCH 2024 and 27 MAY 2024

# BitsAndBytesConfig int-4 config
#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
#)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    #attn_implementation="flash_attention_2", ## BertLMHeadModel does not support Flash Attention 2.0 yet
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    is_decoder=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
model, tokenizer = setup_chat_format(model, tokenizer)

In [ ]:
print(model)

In [ ]:
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
#prompt="What was the first album Beyoncé released as a solo artist?"
prompt="What is the capital of russia?"
#prompt="What are some interesting sites to visit in the Bay Area?"

#prompt = f"Instruct: Find THE answer for the following  question.\n{prompt}\nOutput:\n"
prompt = f"Instruct: Find The answer for the following  question.\n{prompt}"

#prompt="What are some interesting sites to visit in the Bay Area?"

outputs = pipe(prompt, max_new_tokens=256, temperature=0.9,do_sample=True,top_k=50, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

print('Question: %s'%prompt)
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")


In [21]:
index=10
datasetF['Question'][10]
prompt

'Instruct: Find The answer for the following  question.\nWhat is the capital of russia?'

In [24]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cuda')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [22]:
#index=321458
#index=10000
#features: ['id', 'Question', 'Answer'],
index=99
prompt = datasetF['id'][index]
print(prompt)

prompt = datasetF['Question'][index]
print(prompt)

prompt = datasetF['Answer'][index]
print(prompt)

a2480093262
What was the Temperature/Dew Point of the accident no. CEN16LA033?
27°C / 25°C


MODEL GENERATION - ZERO SHOT

In [ ]:
index=99

#features: ['id', 'Question', 'Answer'],
#prompt = dataset[index]['Question']
#summary = dataset[index]['Answer']

prompt = datasetF['Question'][index]
summary = datasetF['Answer'][index]

original_model = model

formatted_prompt = f"Instruct: provide an answer for the following dialog.\n{prompt}\nOutput:\n"
res = original_model.generate(tokenizer.encode(formatted_prompt, return_tensors="pt"), max_length=512, do_sample=False)
output = tokenizer.decode(res[0], skip_special_tokens=True)

input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to('cuda') # Assuming you want to run on GPU
#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to(torch.bfloat16)
#res = model.generate(input_ids, max_length=512, do_sample=False)
#output = tokenizer.decode(res[0], skip_special_tokens=True)

In [25]:
dash_line = '-'.join('' for x in range(100))
print(dash_line)

print(f'Dialogue :\n{formatted_prompt}')
print(dash_line)

print(f'ACTION BY THE AGENT-HUMAN:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
Dialogue :
Instruct: provide an answer for the following dialog.
What was the Temperature/Dew Point of the accident no. CEN16LA033?
Output:

---------------------------------------------------------------------------------------------------
ACTION BY THE AGENT-HUMAN:
27°C / 25°C

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Instruct: provide an answer for the following dialog.
What was the Temperature/Dew Point of the accident no. CEN16LA033?
Output:
y
																																																																																																																																																																																																																																																																																																																																																																						

---------------------------------------------------------------------------------------------------
Dialogue :
Instruct: provide an answer for the following dialog.
What was the date and time of the accident no.ENG20RA030?
Output:

---------------------------------------------------------------------------------------------------
ACTION BY THE AGENT-HUMAN:
February 3, 2020,

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Instruct: provide an answer for the following dialog.
What was the date and time of the accident no.ENG20RA030?
Output:
The date and time of the accident no. ENG20RA030 is not specified in the given information.

In [31]:
print(len(datasetF))

100


In [26]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [27]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [28]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="Mistral-7B-Instruct-v0.1_AviationQA", # 06/06/2024 #dataset_AviationQA

    #num_train_epochs=3,                     # number of training epochs
    num_train_epochs=1,                     # number of training epochs for POC
    per_device_train_batch_size=3,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)

https://huggingface.co/docs/trl/en/sft_trainer

https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Meta_Llama_3_8B_for_text_to_SQL.ipynb

In [29]:
#print(dataset.column_names)
print(datasetF.columns)

Index(['id', 'Question', 'Answer'], dtype='object')


In [30]:
text_field='Bob is a Traveller'

#datasetF = datasetF.add_column('text', [text_field for example in dataset])

# Add a new column named 'text' to the DataFrame
datasetF['text'] = [text_field for _ in range(len(datasetF))]

In [31]:
print(datasetF.columns)

Index(['id', 'Question', 'Answer', 'text'], dtype='object')


In [32]:
print(datasetF['text'][99])

Bob is a Traveller


In [33]:
from trl import SFTTrainer
import datasets

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)


max_seq_length = 3072 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=original_model,
    args=args,


    train_dataset=datasetF_hf,

    #train_dataset=dataset_dialogsum_test,
    dataset_text_field="text", ### added for the summarization dataset
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
        ##"per_device_train_batch_size": 8,
        #"per_device_eval_batch_size": 8,
        #"num_train_epochs": 3.0},
    }
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
#del device
#model.to(torch.device('cuda'))
trainer.train()

# save model
trainer.save_model()

In [35]:
# free the memory again
del model
del original_model
del trainer
torch.cuda.empty_cache()

Test Model and run Inference

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
%cd /content/
#peft_model_id = "./Mistral-7B-text-to-sql"
#/content/Llama-2-7b-hf-text-to-sql-flash-attention-2
#peft_model_id = "./Llama-2-7b-hf-text-to-sql-flash-attention-2"
#peft_model_id = "./Llama-2-7b-hf-dialogsum-test-flash-attention-2"

#peft_model_id = "./OpenMath-Mistral-7B-v0.1-hf-dialogsum-test-flash-attention-2"

#peft_model_id = "./OpenMath-Mistral-7B-v0.1-hf-squad2-flash-attention-2"

peft_model_id = "./Mistral-7B-Instruct-v0.1_AviationQA"

#peft_model_id = "./Mixtral-8x7B-Instruct-v0.1-text-to-sql"
#peft_model_id = "./Smaug-72B-v0.1-text-to-sql" # 04/03/2024
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
modelnew = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline

#KeyError: "Unknown task text-generationn, available tasks are ['audio-classification',
#'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering',
# 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction',
#'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation',
#'ner', 'object-detection', 'question-answering', 'sentiment-analysis',
#'summarization', 'table-question-answering', 'text-classification', 'text-generation',
#'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification',
#'translation', 'video-classification', 'visual-question-answering', 'vqa',
#'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification',
#'zero-shot-object-detection', 'translation_XX_to_YY']"

#Upon looking at the HuggingFacePipeline source code, I found it only supports below pipelines:
#text-generation`, text2text-generation and summarization

#pipe = pipeline("text-generation", model=modelnew, tokenizer=tokenizer)

In [ ]:
pipe = pipeline("text-generation", model=modelnew, tokenizer=tokenizer)

In [ ]:
prompt="What is the capital of russia?"
prompt = f"Instruct: Find The answer for the following  question.\n{prompt}"

outputs = pipe(prompt, max_new_tokens=256, temperature=0.9,do_sample=True,top_k=50, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

print('Question: %s'%prompt)
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")


In [ ]:
print(modelnew)

In [38]:
print(model_id)

mistralai/Mistral-7B-Instruct-v0.1


In [48]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen_after_tunning(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

MODEL GENERATION - AFTER TUNNING

In [ ]:
!pip install accelerate bitsandbytes -q

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

peft_model_id = "./Mistral-7B-Instruct-v0.1_AviationQA"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    ),
    device_map="auto",
    trust_remote_code=True
)
TUNE_model = PeftModel.from_pretrained(model, peft_model_id)
TUNE_model = TUNE_model.merge_and_unload() # Merge LoRA weights and unload the model

index=99
prompt = datasetF['Question'][index]
summary = datasetF['Answer'][index]

formatted_prompt = f"Instruct: provide an answer for the following dialog.\n{prompt}\nOutput:\n"
print(formatted_prompt)
res = gen_after_tunning(TUNE_model,formatted_prompt,1024,)
# ...

output = res[0].split('Output:\n')[1]


In [ ]:
dash_line = '-'.join('' for x in range(100))
print(dash_line)
#print(f'INPUT PROMPT:\n{formatted_prompt}') # for dataset_dialogsum_test
print(f'QUESTION :\n{formatted_prompt}') # for dataset_squad2
print(dash_line)
#print(f'BASELINE HUMAN SUMMARY:\n{summary}\n') # for dataset_dialogsum_test
print(f'ANSWER BY HUMAN:\n{summary}\n') # for dataset_squad2
print(dash_line)
print(f'MODEL GENERATION - AFTER THE TUNNING:\n{output}') #